# Expreimenting with the langraph doc starter

In [19]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from langgraph.graph import END, MessageGraph

model = ChatOpenAI(temperature=0)
graph = MessageGraph()

graph.add_node("oracle", model)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")

runnable = graph.compile()
runnable.invoke(HumanMessage("What is 4 * 5?"))

[HumanMessage(content='What is 4 * 5?', id='f9250710-5984-466a-b0cb-04115b891624'),
 AIMessage(content='4 * 5 equals 20.', response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 15, 'total_tokens': 23}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}, id='0912a85e-61ec-4b3a-a0c0-7f47ef9afc98')]

# Introduction to Conditional edges

In [22]:
import json
from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.messages.base import BaseMessage
from typing import List

@tool
def multiply(first_num: int, second_num: int):
    """
    Multiply two numbers together
    """
    return first_num * second_num

model = ChatOpenAI(temperature=0)
model_with_tools = model.bind(tools=[convert_to_openai_tool(multiply)])

graph = MessageGraph()

def invoke_model(state: List[BaseMessage]):
    return model_with_tools.invoke(state)

graph.add_node("oracle", invoke_model)

def invoke_tool(state: List[BaseMessage]):
    print("State that goes into invoke tool: ", state)
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    multiply_call = None

    for tool_call in tool_calls:
        if tool_call.get("function").get("name") == "multiply":
            multiply_call = tool_call

    if multiply_call is None:
        raise Exception("No adder input found.")

    res = multiply.invoke(
        json.loads(multiply_call.get("function").get("arguments"))
    )
    print("Result on function arguments: ", res)
    print("Tool message: ", ToolMessage(
        tool_call_id=multiply_call.get("id"),
        content=res
    ))
    return ToolMessage(
        tool_call_id=multiply_call.get("id"),
        content=res
    )

graph.add_node("multiply", invoke_tool)

graph.add_edge("multiply", END)

graph.set_entry_point("oracle")

# Router to end if no tool call or use tool if there is tool call
def router(state: List[BaseMessage]):
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "end"
        
# Crucial in order to avoid oracle node being a dead end
graph.add_conditional_edges("oracle", router, {
    "multiply": "multiply",
    "end": END,
})
runnable = graph.compile()

# Maths related question call tool. Especially multiply function
runnable.invoke(HumanMessage("What is 4 * 5?"))

State that goes into invoke tool:  [HumanMessage(content='What is 4 * 5?', id='0bb8dc81-1c43-4909-a4a2-8a90ae20ef5d'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eOuorOFtJFKmNoEcTwf0pJhS', 'function': {'arguments': '{"first_num":4,"second_num":5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 68, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='b3c29ac8-32cd-4443-b85d-2c50ef2d8b27')]
Result on function arguments:  20
Tool message:  content='20' tool_call_id='call_eOuorOFtJFKmNoEcTwf0pJhS'


[HumanMessage(content='What is 4 * 5?', id='0bb8dc81-1c43-4909-a4a2-8a90ae20ef5d'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_eOuorOFtJFKmNoEcTwf0pJhS', 'function': {'arguments': '{"first_num":4,"second_num":5}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 68, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='b3c29ac8-32cd-4443-b85d-2c50ef2d8b27'),
 ToolMessage(content='20', id='7093c1b0-0ecf-4e3c-b2c2-d4a4735c83f3', tool_call_id='call_eOuorOFtJFKmNoEcTwf0pJhS')]

In [23]:
# Non-maths related question does not call tool
runnable.invoke(HumanMessage("What is your name?"))

[HumanMessage(content='What is your name?', id='857c4365-383a-4c6d-b1b2-28d37ed332b6'),
 AIMessage(content='My name is Assistant. How can I assist you today?', response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 65, 'total_tokens': 78}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None}, id='b9587ec4-17e7-4454-b0f7-dbe9e6304e74')]

# Agent Executor from Scratch

First we need to install the packages required

In [1]:
!pip install --quiet -U langchain langchain_openai langchainhub tavily-python

In [7]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OPEN API KEY: ")
os.environ["TAVILY_API_KEY"] = getpass("TAVILY API KEY: ")

OPEN API KEY:  ········
TAVILY API KEY:  ········


Optionally, we can set API key for LangSmith tracing, which will give us best-in-class observability.

In [4]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass("LangSmith API Key: ")

LangSmith API Key:  ········


# Create a Langchain agent

First, we will create the LangChain agent. 

In [23]:
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

prompt = hub.pull("hwchase17/openai-functions-agent")

# print("Prompt: ", prompt, end="\n")

llm = ChatOpenAI(model="gpt-3.5-turbo-1106", streaming=True)

agent_runnable = create_openai_functions_agent(llm, tools, prompt)

# print("Agent runnable: ", agent_runnable, end="\n")

# Define the graph state

We now define the graph state. The state for the traditional LangChain agent has a few attributes:

1. `input`: This is the input string representing the main ask from the user, passed in as input.
2. `chat_history`: This is any previous conversation messages, also passed in as input.
3. `intermediate_steps`: This is list of actions and corresponding observations that the agent takes over time. This is updated each iteration of the agent.
4. `agent_outcome`: This is the response from the agent, either an AgentAction or AgentFinish. The AgentExecutor should finish when this is an AgentFinish, otherwise it should call the requested tools.

In [24]:
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_outcome: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

# Define the nodes

In [28]:
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor
# Create the tool executor
tool_executor = ToolExecutor(tools)

# Define the agent
def run_agent(data):
    agent_outcome = agent_runnable.invoke(data)
    return {"agent_outcome": agent_outcome}

# Define the function to execute tools
def execute_tools(data):
    agent_action = data["agent_outcome"]
    output = tool_executor.invoke(agent_action)
    return {"intermediate_steps": [(agent_action, str(output))]}

# Define logic that will be used to determine which conditional edge to go down
def should_continue(data):
    if isinstance(data["agent_outcome"], AgentFinish):
        return "end"
    else:
        return "continue"
    

# Define the graph

In [29]:
from langgraph.graph import END, StateGraph

# Define a new graph
workflow = StateGraph(AgentState)
# Define two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set entry point as "agent"
workflow.set_entry_point("agent")

# Add a conditional edge
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "action",
        "end": END
    }
)

# Add a normal edge from `tools` to `agent`
# This means after tools is called, `agent` node is called next
workflow.add_edge("action", "agent")

# Compile workflow to runnable
app = workflow.compile()

In [30]:
inputs = {"input": "what is the weather in sf", "chat_history": []}
for s in app.stream(inputs):
    print("Values of each stream: ", list(s.values())[0])
    print("----")

Values of each stream:  {'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'})])}
----
Values of each stream:  {'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"query":"weather in San Francisco"}', 'name': 'tavily_search_results_json'}}, response_metadata={'finish_reason': 'function_call'})]), '[{\'url\': \'https://weather.com/weather/tenday